## 1.1 将序列分解为单独的变量
### 1.1.1 问题
有一个包含N个元素的元组或序列，将它分解为N个单独的变量
### 1.12 解决方案
任何序列(或可迭代对象)都可以通过一个就按单的赋值操作来分解为单独的变量。唯一的要求是变量的总书和结构要与序列相吻合。

In [2]:
p = (4, 5)
x, y = p
print(x)
print(y)

4
5


In [3]:
data = ['ACME', 50, 91.1, (2012, 12, 21)]
name, shares, price, date = data
print(name)
print(shares)
print(price)
print(date)

ACME
50
91.1
(2012, 12, 21)


In [4]:
data = ['ACME', 50, 91.1, (2012, 12, 21)]
name, shares, price, (year, mon, day) = data
print(year)
print(mon)
print(day)

2012
12
21


如果元素的数量不匹配，将得到一个错误提示。

In [2]:
p = (4, 5)
x, y, z = p

ValueError: not enough values to unpack (expected 3, got 2)

### 1.1.3 讨论
只要对象是可迭代的，就可以执行分解操作。包括字符串、文件、迭代器以及生成器。

In [4]:
s = 'Hello'
a, b, c, d, e = s
print(a, b, c, d, e)

H e l l o


做分解操作时，可以丢弃某些特定的值。可以选一个用不到的变量名如`_`，以此来作为要丢弃的值的名称。

In [5]:
data = ['ACME', 50, 91.1, (2012, 12, 21)]
_, shares, price, _ = data
print(shares)
print(price)

50
91.1


## 1.2 从任意长度的可迭代对象中分解元素
### 1.2.1 问题
需要从某个可迭代对象中分解出N个元素，但是这个可迭代对象的长度可能超过N，这回导致出现“分解的值过多(too many values to unpack)”的异常
### 1.2.2 解决方案
Python的`*`表达式可以用来解决这个问题。

In [6]:
# 期末作业成绩等于去掉第一个和最后一个，只对中间剩下的成绩做平均分统计。
from audioop import avg


def drop_first_last(grades):
    first, *middle, last = grades
    return avg(middle)

In [7]:
record = ('Charlie', 'Charlie@gmail.com', '773-555-1212', '847-555-1212')
name, email, *phone_numbers = record
print(name, email, phone_numbers)

Charlie Charlie@gmail.com ['773-555-1212', '847-555-1212']


### 1.2.3 讨论
对于分解未知或任意长度的可迭代对象，这种扩展的分解操作非常有用。通常，这类可迭代独享中会有一些已知的组件或模式(例如，元素1之后的所有内容都是电话号码)。利用*表达式可以分解可迭代对象可以轻松利用这些模式。

In [8]:
records = [
    ('foo', 1, 2),
    ('bar', 'hello'),
    ('foo', 3, 4),
]

def do_foo(x, y):
    print('foo', x, y)

def do_bar(s):
    print('bar', s)

for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


当和某些特定的字符串处理操作相结合，比如做拆分(splitting)操作时，这种*式的语法所支持的分解操作也非常有用。

In [9]:
line = 'nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/flase'
uname, *fields, homedir, sh = line.split(':')
print(uname, homedir, sh)

nobody /var/empty /usr/bin/flase


In [10]:
record = ('ACME', 50, 123.45, (12, 18, 2012))
name, *_, (*_, year) = record
print(name, year)

ACME 2012


## 1.3 保存最后N个元素
### 1.3.1 问题
在迭代或其他形式的处理过程中对最后几项操作做一个有限的历史记录统计
### 1.3.2 解决方案
保存有限的历史记录是`collections.deque`的强项。以下代码对一些列文本行做简单的文本匹配操作，当发现有匹配时就输出当前的匹配行以及最后检查过的N行文本

deque(maxlen=N)创建了一个固定长度的队列。当有新纪录加入而队列已满时会自动移除最老的那条记录。

In [12]:
from collections import deque
q = deque(maxlen=3)
q.append(1)
q.append(2)
q.append(3)
q

deque([1, 2, 3])

In [13]:
q.append(4)
q

deque([2, 3, 4])

In [14]:
q.append(5)
q

deque([3, 4, 5])

当需要一个简单的队列结构时，deque很有帮助。如果不指定队列的大小，就得到了一个无界限的队列，可以在两端执行添加和弹出操作

In [20]:
q = deque()
q.append(1)
q.append(2)
q.append(3)
q

deque([1, 2, 3])

In [21]:
q.appendleft(4)
q

deque([4, 1, 2, 3])

In [22]:
q.pop()
q

deque([4, 1, 2])

In [23]:
q.popleft() # 返回弹出的值

4

读取文件`somefile.txt`，将含有单词`Python`的行输出

In [6]:
from collections import deque

def search(lines, pattern, history=5):
    previous_lines = deque(maxlen=history)
    for line in lines:
        if pattern in line:
            yield line, previous_lines  # 当检测到时，输出当前行line，以及包含前几行的previous_lines
        previous_lines.append(line)

if __name__ == '__main__':
    with open('src/somefile.txt') as f:
        for line, prelines in search(f, 'Python', 5):
            for pline in prelines:
                print(pline, end='')    # 输出记录的前几行，最多保存5行
            print(line, end='') # 输出当前行
            print('-'*20)

hello world
I like Python
--------------------
hello world
I like Python
Life is short, I use Python
--------------------
I like Python
Life is short, I use Python
However, I'm learning cpp now
I love China
one piece
Python is easy to learn
--------------------
Life is short, I use Python
However, I'm learning cpp now
I love China
one piece
Python is easy to learn
I like Python
--------------------


## 1.4 找出最大或最小的N个元素
### 1.4.1 解决方案
`heapq`模块中有两个函数——`nlargest()`和`nsmallest()`

In [7]:
import heapq
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
print(heapq.nlargest(3, nums))  # 输出列表nums中最大的3个元素 [42, 37, 23]
print(heapq.nsmallest(3, nums)) # 输出列表nums中最小的3个元素 [-4, 1, 2]

[42, 37, 23]
[-4, 1, 2]


这两个函数都可以接受一个参数key，运行工作在更加复杂的数据结构之上，自定义排序

In [8]:
portfolio = [
    {'name':'IBM', 'shares':100, 'price':91.1},
    {'name':'AAPL', 'shares':50, 'price':543.22},
    {'name':'FB', 'shares':200, 'price':21.09},
    {'name':'HPQ', 'shares':35, 'price':31.75},
    {'name':'YHOO', 'shares':45, 'price':16.35},
    {'name':'ACME', 'shares':75, 'price':115.65}
]

cheap = heapq.nsmallest(3, portfolio, key=lambda s:s['price'])  # 按'price'排序最小的3个元素
cheap

[{'name': 'YHOO', 'shares': 45, 'price': 16.35},
 {'name': 'FB', 'shares': 200, 'price': 21.09},
 {'name': 'HPQ', 'shares': 35, 'price': 31.75}]

### 1.4.2 讨论
如果正在寻找最大或最小的N个元素，且同集合中元素的总数目相比，N很小，以下方法可以提供更好的性能。这些函数首先在底层将数据转化成列表，且元素会以堆的顺序排列。

堆最重要的特性就是heap[0]总是最小那个的元素。接下来的元素可依次通过`heapq.heappop()`方法找到，该方法会将第一个元素(最小的)弹出，然后以第二小的元素取而代之(复杂度是O(logN)，N代表堆的大小)。要找到第3小的元素，可以这样做：

In [10]:
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
import heapq
heap = list(nums)
heapq.heapify(heap) # heapq.heapify()将列表(list)转换为堆(heap)
heap

[-4, 2, 1, 23, 7, 2, 18, 23, 42, 37, 8]

In [12]:
heapq.heappop(heap) # 最小的元素

-4

In [13]:
heapq.heappop(heap) # 第2小的元素

1

In [14]:
heapq.heappop(heap) # 第3小的元素

2

- 当所要找的元素数量相对较小时，函数`nlargest()`和`nsmallest()`才是最适用的。
- 如果只是简单地找到最小或最大地元素(N=1)，用`min()`和`max()`会更加快。
- 如果N和集合本身地大小差不多大，通常更快地方法是先对集合排序，然后做切片操作。(例如，使用sorted(items)[:N])
- 值得注意的是：`nlargest()`和`nsmallest()`的实际实现会根据使用它们的方式而有所不同，可能会相应做出一些优化措施(比如，当N大小同输入大小很接近时，就会采用排序的方法)

## 1.5 实现优先级队列
### 1.5.1 问题
要实现一个队列，能够以给定的优先级来对元素排序，且每次pop操作时都会返回优先级最高的那个元素。
### 1.5.2 解决方案
下面的类利用`heapq`模块实现了一个简单的优先级队列：

In [1]:
import heapq
class PriorityQueue:

    def __init__(self):
        self._queue = []
        self._index = 0

    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item)) # 向列表_queue插入元素
        self._index += 1

    def pop(self):
        return heapq.heappop(self._queue)[-1]   # 将元素从列表_queue删除，[-1]表示元组中的item

In [2]:
class Item:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return 'Item({!r})'.format(self.name)

q = PriorityQueue()
q.push(Item('foo'), 1)
q.push(Item('bar'), 5)
q.push(Item('spam'), 4)
q.push(Item('grok'), 1)

In [3]:
q.pop()

Item('bar')

In [4]:
q.pop()

Item('spam')

In [5]:
q.pop()

Item('foo')

In [6]:
q.pop()

Item('gork')

第一次执行`pop()`操作时返回的元素具有最高的优先级。当两个元素拥有相同的优先级时，返回的顺序同它们插入到队列时的顺序相同。
### 1.5.3 讨论
在这段代码中，队列以元组(-priority, index, item)的形式组成。
- 把priority取负值是为了让队列能够按元素的优先级从高到低的顺序排序。一般情况下，堆是按从小打到的顺序排序的。
- 变量index的作用是为了将具有相同优先级的元素以适当的顺序排列。通过一个不断递增的索引，元素将以它们入队列时的顺序来排列。

## 1.6 在字典中将键映射到多个值上
### 1.6.1 问题
想要一个能将键(key)映射到多个值的字典(即所谓的一键多值字典[multidict])。
### 1.6.2 解决方案
字典是一个关联容器，每个键都映射到一个单独的值上。如果想让键映射到多个值，需要将这多个值保存到另一个容器如列表或集合中。

要使用列表还是集合完全取决于应用的意图。如果希望保留元素插入的顺序，就用列表。如果希望消除重复元素(且不在意它们的顺序)，就用集合。

为了方便地创建这样地字典，可以利用`collections`模块中的`defaultdict`类。它的一个特点就是会自动初始化第一个值，这样只需关注添加元素即可。

In [8]:
from collections import defaultdict

d =  defaultdict(list)  # 自动创建字典表项以待稍后访问
d['a'].append(1)
d['a'].append(2)
d['b'].append(4)
d

defaultdict(list, {'a': [1, 2], 'b': [4]})

In [9]:
d = defaultdict(set)
d['a'].add(1)
d['a'].add(2)
d['b'].add(4)
d

defaultdict(set, {'a': {1, 2}, 'b': {4}})

In [10]:
# 子啊普通的字典上调用`setdefault()`方法
d = {}  # A regular dictionary
d.setdefault('a', []).append(1)
d.setdefault('a', []).append(2)
d.setdefault('b', []).append(4)
d

{'a': [1, 2], 'b': [4]}

### 1.6.3 讨论
原则上，构建一个一键多值字典是很容易的。但是如果试着自己**对第一个值做初始化操作**，会变得很杂乱。

In [15]:
pairs = {'a':1, 'b':2, 'c':3}

d = {}
for key, value in pairs.items():
    if key not in d:
        d[key] = []     # 普通列表需要对没有的键进行初始化
    d[key].append(value)

d = defaultdict(list)
for key, value in pairs.items():
    d[key].append(value)

## 1.7 让字典保持有序
### 1.7.1 问题
创建一个字典，同时当对字典做迭代或序列化操作时，也能控制其中元素的顺序。
### 1.7.2 解决方案
要控制字典中的元素的顺序，可以使用`collections`模块中的`OrderedDict`类。当对字典做迭代时，它会严格按照元素初始添加的顺序进行。

In [16]:
from collections import OrderedDict
d = OrderedDict()
d['foo'] = 1
d['bar'] = 2
d['spam'] = 3
d['grok'] = 4

for key in d:
    print(key, d[key])

foo 1
bar 2
spam 3
grok 4


当想构建一个映射结构以便稍后对其做序列化或编码成另一种格式时，OrderedDict就显得特别有用。例如，如果想在进行JSON编码时精确控制各字段的顺序，那么只要首先在OrderedDict中构架数据就可以了。

In [17]:
import json
json.dumps(d)

'{"foo": 1, "bar": 2, "spam": 3, "grok": 4}'

### 1.7.3 讨论
OrderedDict内部维护了一个双向链表，他会根据元素加入的顺序来排列键的位置。第一个新加入的元素被放置在链表的末尾。接下来对已存在的键做重新赋值不会改变键的书匈奴。

注意到OrderedDict的大小是普通字典的2倍多，这是由于它额外创建的链表所致。因此，如果打算构建一个涉及大量OederedDict实例的数据结构，需要认真对应用做需求分析。

## 1.8 与字典有关的计算问题
### 1.8.1 问题
在字典上堆数据执行各式各样的计算(比如求最小值、最大值、排序等)
## 1.8.2 解决方案
有如下一个字典在股票名称和对应的价格间做了映射，为了能对字典内容做些有用的计算，通常会利用`zip()`将字典的键和值反转过来。

例如，下面的代码会告诉我们如何找出价格最低和最高的股票。

In [1]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}

min_price = min(zip(prices.values(), prices.keys()))
print(min_price)
max_price = max(zip(prices.values(), prices.keys()))
print(max_price)

(10.75, 'FB')
(612.78, 'AAPL')


In [2]:
# 要堆数据排序只要使用zip()再配合sorted()就可以
prices_sorted = sorted(zip(prices.values(), prices.keys()))
print(prices_sorted)

[(10.75, 'FB'), (37.2, 'HPQ'), (45.23, 'ACME'), (205.55, 'IBM'), (612.78, 'AAPL')]


In [4]:
# 注意zip()创建了一个迭代器，它的内容只能被消费一次。下面代码就是错误的：
prices_sorted = zip(prices.values(), prices.keys())
print(min(prices_sorted))
print(max(prices_sorted))

(10.75, 'FB')


ValueError: max() arg is an empty sequence

### 1.8.3 讨论
如果尝试在字典上执行常见的数据操作，将会发现它们只会处理键，而不是值。
```python
min(prices) # returns 'AAPL' 返回根据键首字母排序结果
max(prices) # returns 'IBM'
```

In [6]:
# 提供一个key参数传递给min()和max()，就能得到最大值和最小值对应的键
print(min(prices, key=lambda k:prices[k])) # 'FB'
print(max(prices, key=lambda k:prices[k])) # 'AAPL'

FB
AAPL


In [8]:
# 如果要得到最小值得话，还需要额外执行一次查找
min_value = prices[min(prices, key=lambda k:prices[k])]
print(min_value)

10.75


利用`zip()`解决方案是通过将字典得键-值对“反转”为值-键对序列。

当在这样得元组上执行比较操作时，值会先进行比较，然后才是键。如果碰到有多个条目拥有相同得value值，那么此时key将用来作为判断结果的依据。

例如，在计算`min()`和`max()`时，如果碰巧value的值相同，则将返回拥有最小或最大key值的那个条目

In [9]:
prices = {'AAA':45.23, 'ZZZ':45.23}
print(min(zip(prices.values(), prices.keys())))
print(max(zip(prices.values(), prices.keys())))

(45.23, 'AAA')
(45.23, 'ZZZ')
